In [1]:
import $ivy.`org.clulab::processors-main:6.1.3`
import $ivy.`org.clulab::processors-corenlp:6.1.3`
import $ivy.`org.clulab::processors-odin:6.1.3`
import $ivy.`org.clulab::processors-modelsmain:6.1.3`
import $ivy.`org.clulab::processors-modelscorenlp:6.1.3`
import $ivy.`com.lihaoyi::pprint:0.5.3`

import org.clulab.odin.ExtractorEngine
import org.clulab.processors.Processor
import org.clulab.processors.clu.CluProcessor
import org.clulab.processors.corenlp.CoreNLPProcessor
import org.clulab.processors.fastnlp.FastNLPProcessor
import org.clulab.processors.shallownlp.ShallowNLPProcessor
import pprint.pprintln

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                                           

import $ivy.$                          


import org.clulab.odin.ExtractorEngine

import org.clulab.processors.Processor

import org.clulab.processors.clu.CluProcessor

import org.clulab.processors.corenlp.CoreNLPProcessor

import org.clulab.processors.fastnlp.FastNLPProcessor

import org.clulab.processors.shallownlp.ShallowNLPProcessor

import pprint.pprintln

In [2]:
// Stanford one
val proc: Processor = new CoreNLPProcessor()
// Also Stanford but Neural Net based (faster).
// val proc: Processor = new FastNLPProcessor()
// CLU lab implementation (faster but less featureful)
// val proc: Processor = new CluProcessor()
val testPhrase = "Hello the world is not flat."
val doc = proc.annotate(testPhrase, true)

Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Reading POS tagger model from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [0.4 sec].
Adding annotator lemma
Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [2.1 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [1.2 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.conll.4class.distsim.crf.ser.gz ... done [1.3 sec].
sutime.binder.1.
Initializing JollyDayHoliday for sutime with classpath:edu/stanford/nlp/models/sutime/jollyday/Holidays_sutime.xml
Reading TokensRegex rules from edu/stanford/nlp/models/sutime/defs.sutime.txt
Reading TokensRegex rules from edu/stanford/nlp/models/sutime/english.sutime.txt
Nov 14, 2017 3:20:58 PM edu.stanford.nlp.ling.tokensregex.CoreMapExpressionExtractor appendRules
INFO: Ignoring inactive rul

proc: Processor = org.clulab.processors.corenlp.CoreNLPProcessor@4f5f643c
testPhrase: String = "Hello the world is not flat."
doc: org.clulab.processors.Document = org.clulab.processors.corenlp.CoreNLPDocument@171b93f4

In [3]:
import org.clulab.odin.{EventMention, Mention, RelationMention, TextBoundMention}
import org.clulab.processors.{Document, Sentence}

object NLPPrinter {

  def displayMentions(mentions: Seq[Mention], doc: Document): Unit = {
    val mentionsBySentence = mentions groupBy (_.sentence) mapValues (_.sortBy(_.start)) withDefaultValue Nil
    for ((s, i) <- doc.sentences.zipWithIndex) {
      println(s"sentence #$i")
      println(s.getSentenceText)
      println("Tokens: " + (s.words.indices, s.words, s.tags.get).zipped.mkString(", "))
      printSyntacticDependencies(s)
      println

      val sortedMentions     = mentionsBySentence(i).sortBy(_.label)
      val (events, entities) = sortedMentions.partition(_ matches "Event")
      val (tbs, rels)        = entities.partition(_.isInstanceOf[TextBoundMention])
      val sortedEntities     = tbs ++ rels.sortBy(_.label)
      println("entities:")
      sortedEntities foreach displayMention

      println
      println("events:")
      events foreach displayMention
      println("=" * 50)
    }
  }

  def printSyntacticDependencies(s: Sentence): Unit = {
    if (s.dependencies.isDefined) {
      println(s.dependencies.get.toString)
    }
  }

  def displayMention(mention: Mention) {
    val boundary = s"\t${"-" * 30}"
    println(s"${mention.labels} => ${mention.text}")
    println(boundary)
    println(s"\tRule => ${mention.foundBy}")
    val mentionType = mention.getClass.toString.split("""\.""").last
    println(s"\tType => $mentionType")
    println(boundary)
    mention match {
      case tb: TextBoundMention =>
        println(s"\t${tb.labels.mkString(", ")} => ${tb.text}")
      case em: EventMention =>
        println(s"\ttrigger => ${em.trigger.text}")
        displayArguments(em)
      case rel: RelationMention =>
        displayArguments(rel)
      case _ => ()
    }
    println(s"$boundary\n")
  }

  def displayArguments(b: Mention): Unit = {
    b.arguments foreach {
      case (argName, ms) =>
        ms foreach { v =>
          println(s"\t$argName ${v.labels.mkString("(", ", ", ")")} => ${v.text}")
        }
    }
  }
}


import org.clulab.odin.{EventMention, Mention, RelationMention, TextBoundMention}

import org.clulab.processors.{Document, Sentence}


defined object NLPPrinter

In [5]:
import java.io.File

val rules = scala.io.Source.fromFile(new File("../core/src/main/resources/grammars/claims/master.yml")).mkString
val extractor       = ExtractorEngine(rules)

val phrase = "hello"
val doc = proc.annotate(phrase)
val mentions = extractor.extractFrom(doc)

NLPPrinter.displayMentions(mentions, doc)
mentions.find(_.label.contains("Date")).map(f => pprintln(f.tokenInterval, height=Int.MaxValue))
pprintln(doc.sentences(0).norms)

sentence #0
hello
Tokens: (0,hello,UH)
roots: 0
outgoing:
incoming:


entities:

events:
List(greet, Intent, Event) => hello
	------------------------------
	Rule => greet-2
	Type => TextBoundMention
	------------------------------
	greet, Intent, Event => hello
	------------------------------

Some(Array("O"))


import java.io.File


rules: String = """
#resources:
#  # use sample of w2v embeddings generated from gigaword
#  embeddings: classpath:org/clulab/odin/embeddings/w2v-gigaword-sample.txt

taxonomy:
  - Entity:
    - Subject
    - Object
    - PossiblePerson:
      - Person
      - Pronoun
...
extractor: ExtractorEngine = org.clulab.odin.ExtractorEngine@4730a1f1
phrase: String = "hello"
doc: Document = org.clulab.processors.corenlp.CoreNLPDocument@74f52e3c
mentions: Seq[Mention] = Vector(org.clulab.odin.TextBoundMention@335de76e)
res4_7: Option[Unit] = None

In [4]:
import $ivy.`org.typelevel::spire-extras:0.14.1`

import $ivy.$                                   

In [5]:
import spire.algebra._
import spire.math._
import spire.implicits._

val v = Vector(
  ("coverage_query", 0.2864827513694763),
  ("loss_cause", 0.2397916465997696),
  ("billing_query", 0.15513364970684052),
  ("policy_query", 0.14402523636817932),
  ("policy_change_request", 0.03895655646920204),
  ("bad_feedback", 0.0363447479903698),
  ("greet", 0.030883004888892174),
  ("initiate_complaint", 0.028019487857818604),
  ("claim_initiate", 0.026078691706061363),
  ("goodbye", 0.014284259639680386)
)



// def stdDev[T: Numeric](a: Vector[T]): T = {
//     a.normalize.map(_.pow(2)).qsum/a.length
// }

// v.map(_._2).normalize.map(_.pow(2)).qsum/v.length
val h = v.map(_._2)
val mean = h.sum/h.length
val meanVector = h.map(_ - mean)
(h.map( _ - mean).map(t => t*t).sum/h.length).sqrt
(h.normalize.map(_.pow(2)).qsum/h.length).sqrt
h.map(_ - mean).map(_.pow(2)).qmean.sqrt

import spire.algebra._

import spire.math._

import spire.implicits._


v: Vector[(String, Double)] = Vector(
  ("coverage_query", 0.2864827513694763),
  ("loss_cause", 0.2397916465997696),
  ("billing_query", 0.15513364970684052),
  ("policy_query", 0.14402523636817932),
  ("policy_change_request", 0.03895655646920204),
  ("bad_feedback", 0.0363447479903698),
  ("greet", 0.030883004888892174),
  ("initiate_complaint", 0.028019487857818604),
  ("claim_initiate", 0.026078691706061363),
  ("goodbye", 0.014284259639680386)
)
h: Vector[Double] = Vector(
  0.2864827513694763,
  0.2397916465997696,
  0.15513364970684052,
  0.14402523636817932,
  0.03895655646920204,
  0.0363447479903698,
  0.030883004888892174,
  0.028019487857818604,
  0.026078691706061363,
  0.014284259639680386
)
mean: Double = 0.10000000325962902
meanVector: Vector[Double] = Vector(
  0.1864827481098473,
  0.13979164334014058,
  0.0551336464472115,
  0.044025233108550305,
  -0.061043446790426975,
  -0.06365525526925922,
